In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.6MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


('번호', '앨범번호', '앨범이름', '가수리스트(띄어쓰기구분)', '노래제목', '장르', '태그')

In [4]:
def get_list_df(item, col, day,genre=''):
  #노래 번호 얻기
  try:
    get_item = item.find('div', {'class' : 'rank01'}).find('a')
    song_num = get_item['href'].split(',')[-1][:-2]
  except:
    print('song_num err')
    song_num = ''

  #노래제목
  try:
    song_title = get_item.text
  except:
    print('song_title err')
    song_title = ''

  #가수, 가수번호
  try:
    get_item = item.find('div', {'class' : 'rank02'}).find_all('a')
    artist_set = set()
    artist_list = []
    artist_num_list = []
    for artist_item in get_item:
      artist_tuple = (artist_item.text, artist_item['href'].split("'")[-2])
      artist_set.add(artist_tuple)
    
    for set_item in artist_set:
        artist_list.append(set_item[0])
        artist_num_list.append(set_item[1])
    artist = (' , ').join(artist_list)
    artist_num = (' ').join(artist_num_list)
  except:
    print('artist err')
    artist = ''
    artist_num=''

  #앨범이름
  try:
    get_item = item.find('div', {'class' : 'rank03'}).find('a')
    album_title = get_item.text
  except:
    print('al_title err')
    album_title = ''

  #앨범번호
  try:
    al_href = get_item['href']
    album_num = al_href.split("'")[-2]
  except:
    print('album_num err')
    album_num = ''

  #좋아요 수
  try:
    get_item = item.find('span', {'class' : 'cnt'})
    luv = get_item.text.split('\n')[-1]
    if (genre == '국내종합'):
      print(get_item.text.split('\n')[-1])
      print('--------------------------------')
  except:
    print('luv err')
    luv=''
                   #('날짜', '번호', '앨범번호', '앨범이름', '가수리스트(띄어쓰기구분)', '가수번호', '노래제목', '장르', '좋아요', '태그')
  return pd.DataFrame([day, song_num, album_num,  album_title, artist, artist_num, song_title, genre, luv ,'#인기순위 '+ '#일간순위 ' + '#'+genre], index=col).transpose()


In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://www.melon.com/chart/day/index.htm?classCd='
genre_index = ['GN0000', 'DM0000', 'AB0000', 'GN0100', 'GN0200', 'GN0300', 'GN0400', 'GN0500', 'GN0600', 'GN0700', 'GN0800', 'GN0900', 'GN1000', 'GN1100', 'GN1200', 'GN1300', 'GN1400', 'GN1500', 'GN1600', 'GN1700', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400'] 
genres = ['장르종합', '국내종합', '해외종합', '발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'J-POP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']
genre_dict = dict(zip(genre_index, genres))

col = ['날짜', '번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
df = pd.DataFrame(columns=col)

day = '20210714'

for genre in genre_index:
  base_url = url+genre

  driver.get(base_url)
  time.sleep(5)
  body = driver.find_element_by_css_selector('body')
  res = driver.page_source
  soup = bs(res, "html.parser")
  table_body = soup.find('tbody')
  table_item = table_body.find_all('tr')

  for item in table_item:
    df = pd.concat([df, get_list_df(item, col, day, genre_dict[genre])], axis=0)
  print(genre_dict[genre], "done,", (genre_index.index(genre)+1), "/", len(genre_index))

df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간랭킹/'+day+'일간랭킹_모든항목'+'.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


장르종합 done, 1 / 27
107,603
--------------------------------
164,513
--------------------------------
223,962
--------------------------------
116,563
--------------------------------
110,520
--------------------------------
67,936
--------------------------------
126,772
--------------------------------
61,217
--------------------------------
81,353
--------------------------------
128,224
--------------------------------
282,283
--------------------------------
60,950
--------------------------------
417,197
--------------------------------
204,942
--------------------------------
110,824
--------------------------------
87,958
--------------------------------
292,202
--------------------------------
54,831
--------------------------------
124,067
--------------------------------
55,805
--------------------------------
27,852
--------------------------------
95,339
--------------------------------
114,048
--------------------------------
96,283
--------------------------------
95,546
-

#테스트

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://www.melon.com/chart/day/index.htm?classCd='
genre_index = ['GN0000', 'DM0000', 'AB0000', 'GN0100', 'GN0200', 'GN0300', 'GN0400', 'GN0500', 'GN0600', 'GN0700', 'GN0800', 'GN0900', 'GN1000', 'GN1100', 'GN1200', 'GN1300', 'GN1400', 'GN1500', 'GN1600', 'GN1700', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400'] 
genres = ['장르종합', '국내종합', '해외종합', '발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'J-POP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']
genre_dict = dict(zip(genre_index, genres))

col = ['날짜', '번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
df = pd.DataFrame(columns=col)

day = '20210711'

genre = genre_index[1]
base_url = url+genre

driver.get(base_url)
body = driver.find_element_by_css_selector('body')
res = driver.page_source
soup = bs(res, "html.parser")
table_body = soup.find('tbody')
table_item = table_body.find_all('tr')

for item in table_item:
  df = pd.concat([df, get_list_df(item, col, day, genre_dict[genre])], axis=0)
print(genre_dict[genre], "done,", (genre_index.index(genre)+1), "/", len(genre_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


101,183
--------------------------------
161,360
--------------------------------
221,897
--------------------------------
103,777
--------------------------------
102,291
--------------------------------
124,860
--------------------------------
59,854
--------------------------------
59,193
--------------------------------
79,951
--------------------------------
127,201
--------------------------------
281,809
--------------------------------
59,455
--------------------------------
416,604
--------------------------------
204,316
--------------------------------
109,747
--------------------------------
52,431
--------------------------------
86,253
--------------------------------
123,572
--------------------------------
291,807
--------------------------------
55,058
--------------------------------
26,548
--------------------------------
94,279
--------------------------------
113,123
--------------------------------
95,818
--------------------------------
94,653
-------------------

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://www.melon.com/chart/day/index.htm?classCd='
genre_index = ['GN0000', 'DM0000', 'AB0000', 'GN0100', 'GN0200', 'GN0300', 'GN0400', 'GN0500', 'GN0600', 'GN0700', 'GN0800', 'GN0900', 'GN1000', 'GN1100', 'GN1200', 'GN1300', 'GN1400', 'GN1500', 'GN1600', 'GN1700', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400'] 
genre_index_test = ['GN0000', 'DM0000']
genres = ['장르종합', '국내종합', '해외종합', '발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'J-POP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']
genre_dict = dict(zip(genre_index, genres))

col = ['날짜', '번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
df = pd.DataFrame(columns=col)

day = '20210710'

for genre in genre_index_test:
  base_url = url+genre

  driver.get(base_url)
  time.sleep(5)
  body = driver.find_element_by_css_selector('body')
  res = driver.page_source
  soup = bs(res, "html.parser")
  table_body = soup.find('tbody')
  table_item = table_body.find_all('tr')

  for item in table_item:
    df = pd.concat([df, get_list_df(item, col, day, genre_dict[genre])], axis=0)
  print(genre_dict[genre], "done,", (genre_index.index(genre)+1), "/", len(genre_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


장르종합 done, 1 / 27
97,489
--------------------------------
159,848
--------------------------------
220,864
--------------------------------
123,867
--------------------------------
98,102
--------------------------------
58,967
--------------------------------
93,374
--------------------------------
79,203
--------------------------------
126,686
--------------------------------
53,912
--------------------------------
281,585
--------------------------------
58,687
--------------------------------
416,216
--------------------------------
203,989
--------------------------------
109,216
--------------------------------
51,305
--------------------------------
85,430
--------------------------------
123,364
--------------------------------
291,620
--------------------------------
54,628
--------------------------------
25,666
--------------------------------
93,777
--------------------------------
112,598
--------------------------------
95,579
--------------------------------
94,216
----